In [1]:
pip install xgboost lightgbm

Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com
Note: you may need to restart the kernel to use updated packages.


In [2]:
import boto3
import pandas as pd; pd.set_option('display.max_columns', 100)
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

from scipy.stats import rankdata
from sklearn.tree import DecisionTreeRegressor, plot_tree
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import KFold, train_test_split, GridSearchCV, StratifiedKFold
from sklearn.metrics import mean_squared_error, roc_auc_score
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import HistGradientBoostingRegressor, RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier

s3 = boto3.resource('s3')
bucket_name = 'analytics-data-science-competitions'
bucket = s3.Bucket(bucket_name)

file_key_1 = 'Tabular-Playground-Series/Tabular-Playground-Jan-2023-2/train.csv'
file_key_2 = 'Tabular-Playground-Series/Tabular-Playground-Jan-2023-2/test.csv'
file_key_3 = 'Tabular-Playground-Series/Tabular-Playground-Jan-2023-2/sample_submission.csv'

bucket_object_1 = bucket.Object(file_key_1)
file_object_1 = bucket_object_1.get()
file_content_stream_1 = file_object_1.get('Body')

bucket_object_2 = bucket.Object(file_key_2)
file_object_2 = bucket_object_2.get()
file_content_stream_2 = file_object_2.get('Body')

bucket_object_3 = bucket.Object(file_key_3)
file_object_3 = bucket_object_3.get()
file_content_stream_3 = file_object_3.get('Body')

## Reading data files
train = pd.read_csv(file_content_stream_1)
test = pd.read_csv(file_content_stream_2)
submission = pd.read_csv(file_content_stream_3)

In [3]:
from bisect import bisect_left

def rank_test(train_preds, test_preds):
    # rank train preds
    train_rank = rankdata(train_preds)
    preds_rank_zip = list(zip(train_preds, train_rank))

    # sort zipped list by first key
    preds_rank_zip.sort(key = lambda x: x[0])

    # unzip the sorted zipped list
    train_preds, train_rank = zip(*preds_rank_zip)

    # use the closest prediction to get the closets rank
    return np.array([
        train_rank[
            bisect_left(train_preds, x)
        ] for x in test_preds
    ])

In [3]:
data1 = pd.read_csv('test_10_by_10x-Lasso_0872729_2023-01-11-00-22.csv')
data2 = pd.read_csv('ensemble_submission.csv')

# rank_test(data1['stroke'])

In [10]:
data1 = pd.read_csv('test_10_by_10x-Lasso_0872729_2023-01-11-00-22.csv')
data2 = pd.read_csv('RF_submission.csv')

data3 = data1.copy()
data3['stroke'] = (data1['stroke'] + data2['stroke']) / 2
data3.head()

,id,stroke
0,15304,0.276274
1,15305,0.384600
2,15306,0.250294
3,15307,0.281817
4,15308,0.260088


In [13]:
data3.to_csv('ensemble_1_submission.csv', index = False)

In [12]:
data1 = pd.read_csv('test_10_by_10x-Lasso_0872729_2023-01-11-00-22.csv')
data2 = pd.read_csv('TF_submission.csv')
data3 = pd.read_csv('another_submission.csv')


In [13]:
data4 = data1.copy()

data4['stroke'] = (0.872729*data1['stroke'] + 0.87841*data2['stroke'] + 0.88528*data3['stroke']) / (0.872729 + 0.87841 + 0.88528)
data4.head()

,id,stroke
0,15304,0.197866
1,15305,0.366678
2,15306,0.165904
3,15307,0.216273
4,15308,0.171791


In [14]:
data4.to_csv('ensemble_5_submission.csv', index = False)

In [4]:
def corrected_age(X):
    return  (         X['age']
             +5.6   * X['hypertension']
             +6     * X['smoking_status_smokes']
             +0.7   * X['smoking_status_Unknown']
             +0.9   * X['heart_disease']
             +0.8   * X['gender_Male']
             -1.2   * X['ever_married_Yes']
             -8.6   * X['work_type_Never_worked']
             -3.5   * X['work_type_Self-employed']
             +0.087 * X['avg_glucose_level']
             +0.23  * X['bmi']
            )

In [5]:
submission['stroke'] = corrected_age(pd.get_dummies(test))
submission.head()

,id,stroke
0,15304,71.36298
1,15305,88.72422
2,15306,19.20864
3,15307,76.34588
4,15308,54.19305


In [6]:
submission.to_csv('ambrosm_submission.csv', index = False)

# Logistic Modeling

In [3]:
train_logit = train.copy()
test_logit = test.copy()

## Defining scaler
scaler = MinMaxScaler()

## Defining inputs and target
train_dummies = pd.get_dummies(train_logit[['gender', 'ever_married', 'work_type', 'Residence_type', 'smoking_status']])
train_dummies = train_dummies.drop(columns = ['gender_Other', 'ever_married_No', 'work_type_children', 'Residence_type_Urban', 'smoking_status_Unknown'])

X = train_logit.drop(columns = ['id', 'gender', 'ever_married', 'work_type', 'Residence_type', 'smoking_status', 'stroke'], axis = 1)
X = pd.concat([X, train_dummies], axis = 1)
X = pd.DataFrame(scaler.fit_transform(X), columns = X.columns)
Y = train_logit['stroke']

test_dummies = pd.get_dummies(test_logit[['gender', 'ever_married', 'work_type', 'Residence_type', 'smoking_status']])
test_dummies = test_dummies.drop(columns = ['gender_Other', 'ever_married_No', 'work_type_children', 'Residence_type_Urban', 'smoking_status_Unknown'])

test_logit = test_logit.drop(columns = ['id', 'gender', 'ever_married', 'work_type', 'Residence_type', 'smoking_status'], axis = 1)
test_logit = pd.concat([test_logit, test_dummies], axis = 1)
test_logit = pd.DataFrame(scaler.fit_transform(test_logit), columns = test_logit.columns)

logit_md = LogisticRegression(C = 10, penalty = 'l2', solver = 'sag', max_iter = 1000).fit(X, Y)
logit_pred = logit_md.predict_proba(test_logit)[:, 1]

submission['stroke'] = logit_pred
submission.head()

,id,stroke
0,15304,0.035890
1,15305,0.145910
2,15306,0.000305
3,15307,0.047811
4,15308,0.006601


In [33]:
submission.to_csv('logistic_full_correct_submission.csv', index = False)

In [17]:
logit_preds = list()

for i in range(1000):
    
    logit_md = LogisticRegression(C = 0.01, penalty = 'l1', solver = 'liblinear', max_iter = 1000, random_state = i).fit(X, Y)
    logit_preds.append(logit_md.predict_proba(test_logit)[:, 1])
    
logit_preds = pd.DataFrame(logit_preds)
submission['stroke'] = logit_preds.apply(np.mean, axis = 0)
submission.head()

,id,stroke
0,15304,0.057258
1,15305,0.068916
2,15306,0.026798
3,15307,0.056442
4,15308,0.039867


In [18]:
submission.describe()

,id,stroke
count,10204.000000,10204.000000
mean,20405.500000,0.048114
std,2945.785407,0.014448
min,15304.000000,0.024914
25%,17854.750000,0.036516
50%,20405.500000,0.047130
75%,22956.250000,0.057258
max,25507.000000,0.081591


In [10]:
submission.to_csv('logistic_0_0_0_1_full_1000_submission.csv', index = False)

# Random Forest Modeling

In [2]:
train_RF = train.copy()
test_RF = test.copy()

## Defining inputs and target
train_dummies = pd.get_dummies(train_RF[['gender', 'ever_married', 'work_type', 'Residence_type', 'smoking_status']])
train_dummies = train_dummies.drop(columns = ['gender_Other', 'ever_married_No', 'work_type_children', 'Residence_type_Urban', 'smoking_status_Unknown'])

X = train_RF.drop(columns = ['id', 'gender', 'ever_married', 'work_type', 'Residence_type', 'smoking_status', 'stroke'], axis = 1)
X = pd.concat([X, train_dummies], axis = 1)
Y = train_RF['stroke']

test_dummies = pd.get_dummies(test_RF[['gender', 'ever_married', 'work_type', 'Residence_type', 'smoking_status']])
test_dummies = test_dummies.drop(columns = ['gender_Other', 'ever_married_No', 'work_type_children', 'Residence_type_Urban', 'smoking_status_Unknown'])

test_RF = test_RF.drop(columns = ['id', 'gender', 'ever_married', 'work_type', 'Residence_type', 'smoking_status'], axis = 1)
test_RF = pd.concat([test_RF, test_dummies], axis = 1)

RF_preds = list()

for i in range(1000):
    print(i)
    RF_md = RandomForestClassifier(max_depth = 7, min_samples_leaf = 5, min_samples_split = 2, n_estimators = 300).fit(X, Y)
    RF_preds.append(RF_md.predict_proba(test_RF)[:, 1])
    
RF_preds = pd.DataFrame(RF_preds)
submission['stroke'] = RF_preds.apply(np.mean, axis = 0)
submission.head()

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

,id,stroke
0,15304,0.035033
1,15305,0.228464
2,15306,0.001450
3,15307,0.052045
4,15308,0.019863


In [3]:
submission.to_csv('RF_full_1000_submission.csv', index = False)

# XGBoost Modeling

In [11]:
train_XGB = train.copy()
test_XGB = test.copy()

## Defining inputs and target
train_dummies = pd.get_dummies(train_XGB[['gender', 'ever_married', 'work_type', 'Residence_type', 'smoking_status']])
train_dummies = train_dummies.drop(columns = ['gender_Other', 'ever_married_No', 'work_type_children', 'Residence_type_Urban', 'smoking_status_Unknown'])

X = train_XGB.drop(columns = ['id', 'gender', 'ever_married', 'work_type', 'Residence_type', 'smoking_status', 'stroke'], axis = 1)
X = pd.concat([X, train_dummies], axis = 1)
Y = train_XGB['stroke']

XGBoost_param_grid = {'n_estimators': [300, 500],
                      'max_depth': [5, 7],
                      'min_child_weight': [5, 7, 10],
                      'learning_rate': [0.01],
                      'gamma': [0.3, 0.1],
                      'subsample': [0.8, 1],
                      'colsample_bytree': [0.8, 0.9]}

## Performing grid search with 5 folds
XGBoost_grid_search = GridSearchCV(XGBClassifier(), XGBoost_param_grid, cv = 5, scoring = 'roc_auc', n_jobs = -1, verbose = 1).fit(X, Y)

## Extracting the best parameters
best_params = XGBoost_grid_search.best_params_
print('The optimal hyper-parameters are:', best_params)

Fitting 5 folds for each of 96 candidates, totalling 480 fits
The optimal hyper-parameters are: {'colsample_bytree': 0.8, 'gamma': 0.3, 'learning_rate': 0.01, 'max_depth': 5, 'min_child_weight': 10, 'n_estimators': 500, 'subsample': 0.8}


In [14]:
train_XGB = train.copy()
test_XGB = test.copy()

## Defining inputs and target
train_dummies = pd.get_dummies(train_XGB[['gender', 'ever_married', 'work_type', 'Residence_type', 'smoking_status']])
train_dummies = train_dummies.drop(columns = ['gender_Other', 'ever_married_No', 'work_type_children', 'Residence_type_Urban', 'smoking_status_Unknown'])

X = train_XGB.drop(columns = ['id', 'gender', 'ever_married', 'work_type', 'Residence_type', 'smoking_status', 'stroke'], axis = 1)
X = pd.concat([X, train_dummies], axis = 1)
Y = train_XGB['stroke']

test_dummies = pd.get_dummies(test_XGB[['gender', 'ever_married', 'work_type', 'Residence_type', 'smoking_status']])
test_dummies = test_dummies.drop(columns = ['gender_Other', 'ever_married_No', 'work_type_children', 'Residence_type_Urban', 'smoking_status_Unknown'])

test_XGB = test_XGB.drop(columns = ['id', 'gender', 'ever_married', 'work_type', 'Residence_type', 'smoking_status'], axis = 1)
test_XGB = pd.concat([test_XGB, test_dummies], axis = 1)

cv_scores, roc_auc_scores = list(), list()
preds = list()

## Running 5 times CV
for i in range(5):
    
    skf = StratifiedKFold(n_splits = 5, random_state = 42, shuffle = True)
    
    for train_ix, test_ix in skf.split(X, Y):
        
        ## Splitting the data 
        X_train, X_test = X.iloc[train_ix], X.iloc[test_ix]
        Y_train, Y_test = Y.iloc[train_ix], Y.iloc[test_ix]
    
        ## Building RF model
        XGB_md = XGBClassifier(colsample_bytree = 0.8, 
                               gamma = 0.3, 
                               learning_rate = 0.01, 
                               max_depth = 5, 
                               min_child_weight = 10, 
                               n_estimators = 500, 
                               subsample = 0.8).fit(X_train, Y_train)
        
        ## Predicting on X_test and test
        XGB_pred_1 = XGB_md.predict_proba(X_test)[:, 1]
        XGB_pred_2 = XGB_md.predict_proba(test_XGB)[:, 1]
        
        ## Computing roc-auc score
        roc_auc_scores.append(roc_auc_score(Y_test, XGB_pred_1))
        preds.append(XGB_pred_2)
        
    cv_scores.append(np.mean(roc_auc_scores))
    
print('The roc-auc score over 5-folds (run 5 times) is:', np.mean(cv_scores))

The roc-auc score over 5-folds (run 5 times) is: 0.8853855128396434


# LightGBM Modeling

In [20]:
train_lgb = train.copy()
test_lgb = test.copy()

## Defining inputs and target
train_dummies = pd.get_dummies(train_lgb[['gender', 'ever_married', 'work_type', 'Residence_type', 'smoking_status']])
train_dummies = train_dummies.drop(columns = ['gender_Other', 'ever_married_No', 'work_type_children', 'Residence_type_Urban', 'smoking_status_Unknown'])

X = train_lgb.drop(columns = ['id', 'gender', 'ever_married', 'work_type', 'Residence_type', 'smoking_status', 'stroke'], axis = 1)
X = pd.concat([X, train_dummies], axis = 1)
Y = train_lgb['stroke']

LightGBM_param_grid = {'n_estimators': [1000],
                       'max_depth': [7, 9],
                       'learning_rate': [0.001, 0.01],
                       'num_leaves': [20, 40],
                       'lambda_l1': [1, 3],
                       'lambda_l2': [1, 3],
                       'bagging_fraction': [0.7, 0.8],
                       'feature_fraction': [0.7, 0.8]
                      }

## Performing grid search with 5 folds
LightGBM_grid_search = GridSearchCV(LGBMClassifier(), LightGBM_param_grid, cv = 5, scoring = 'roc_auc', n_jobs = -1, verbose = 1).fit(X, Y)

## Extracting the best parameters
best_params = LightGBM_grid_search.best_params_
print('The optimal hyper-parameters are:', best_params)

Fitting 5 folds for each of 128 candidates, totalling 640 fits
[LightGBM] [Warning] lambda_l1 is set=1, reg_alpha=0.0 will be ignored. Current value: lambda_l1=1
[LightGBM] [Warning] feature_fraction is set=0.7, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.7
[LightGBM] [Warning] lambda_l2 is set=1, reg_lambda=0.0 will be ignored. Current value: lambda_l2=1
[LightGBM] [Warning] bagging_fraction is set=0.7, subsample=1.0 will be ignored. Current value: bagging_fraction=0.7
[LightGBM] [Warning] lambda_l1 is set=1, reg_alpha=0.0 will be ignored. Current value: lambda_l1=1
[LightGBM] [Warning] feature_fraction is set=0.7, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.7
[LightGBM] [Warning] lambda_l2 is set=1, reg_lambda=0.0 will be ignored. Current value: lambda_l2=1
[LightGBM] [Warning] bagging_fraction is set=0.7, subsample=1.0 will be ignored. Current value: bagging_fraction=0.7
[LightGBM] [Warning] lambda_l1 is set=1, reg_alpha=0.0 wi

In [21]:
best_params = LightGBM_grid_search.best_params_
print('The optimal hyper-parameters are:', best_params)

The optimal hyper-parameters are: {'bagging_fraction': 0.7, 'feature_fraction': 0.7, 'lambda_l1': 3, 'lambda_l2': 3, 'learning_rate': 0.01, 'max_depth': 7, 'n_estimators': 1000, 'num_leaves': 20}


In [25]:
train_lgb = train.copy()
test_lgb = test.copy()

## Defining inputs and target
train_dummies = pd.get_dummies(train_lgb[['gender', 'ever_married', 'work_type', 'Residence_type', 'smoking_status']])
train_dummies = train_dummies.drop(columns = ['gender_Other', 'ever_married_No', 'work_type_children', 'Residence_type_Urban', 'smoking_status_Unknown'])

X = train_lgb.drop(columns = ['id', 'gender', 'ever_married', 'work_type', 'Residence_type', 'smoking_status', 'stroke'], axis = 1)
X = pd.concat([X, train_dummies], axis = 1)
Y = train_lgb['stroke']

test_dummies = pd.get_dummies(test_lgb[['gender', 'ever_married', 'work_type', 'Residence_type', 'smoking_status']])
test_dummies = test_dummies.drop(columns = ['gender_Other', 'ever_married_No', 'work_type_children', 'Residence_type_Urban', 'smoking_status_Unknown'])

test_lgb = test_lgb.drop(columns = ['id', 'gender', 'ever_married', 'work_type', 'Residence_type', 'smoking_status'], axis = 1)
test_lgb = pd.concat([test_lgb, test_dummies], axis = 1)

cv_scores, roc_auc_scores = list(), list()
preds = list()

## Running 5 times CV
for i in range(5):
    
    skf = StratifiedKFold(n_splits = 5, random_state = 42, shuffle = True)
    
    for train_ix, test_ix in skf.split(X, Y):
        
        ## Splitting the data 
        X_train, X_test = X.iloc[train_ix], X.iloc[test_ix]
        Y_train, Y_test = Y.iloc[train_ix], Y.iloc[test_ix]
    
        ## Building RF model
        lgb_md = LGBMClassifier(n_estimators = 1000,
                                max_depth = 7,
                                learning_rate = 0.01,
                                num_leaves = 20,
                                lambda_l1 = 3,
                                lambda_l2 = 3,
                                bagging_fraction = 0.7,
                                feature_fraction = 0.7).fit(X_train, Y_train)
        
        ## Predicting on X_test and test
        lgb_pred_1 = lgb_md.predict_proba(X_test)[:, 1]
        lgb_pred_2 = lgb_md.predict_proba(test_lgb)[:, 1]
        
        ## Computing roc-auc score
        roc_auc_scores.append(roc_auc_score(Y_test, lgb_pred_1))
        preds.append(lgb_pred_2)
        
    cv_scores.append(np.mean(roc_auc_scores))
    
print('The roc-auc score over 5-folds (run 5 times) is:', np.mean(cv_scores))

[LightGBM] [Warning] lambda_l1 is set=3, reg_alpha=0.0 will be ignored. Current value: lambda_l1=3
[LightGBM] [Warning] feature_fraction is set=0.7, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.7
[LightGBM] [Warning] lambda_l2 is set=3, reg_lambda=0.0 will be ignored. Current value: lambda_l2=3
[LightGBM] [Warning] bagging_fraction is set=0.7, subsample=1.0 will be ignored. Current value: bagging_fraction=0.7
[LightGBM] [Warning] lambda_l1 is set=3, reg_alpha=0.0 will be ignored. Current value: lambda_l1=3
[LightGBM] [Warning] feature_fraction is set=0.7, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.7
[LightGBM] [Warning] lambda_l2 is set=3, reg_lambda=0.0 will be ignored. Current value: lambda_l2=3
[LightGBM] [Warning] bagging_fraction is set=0.7, subsample=1.0 will be ignored. Current value: bagging_fraction=0.7
[LightGBM] [Warning] lambda_l1 is set=3, reg_alpha=0.0 will be ignored. Current value: lambda_l1=3
[LightGBM] [Warning] 

In [ ]:
0.8826822779125866